In [28]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

DEBUG:matplotlib:matplotlib data path: /home/anl/git/satelites/venv/lib/python3.10/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/anl/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/anl/.cache/matplotlib
DEBUG:matplotlib.font_manager:font search path [PosixPath('/home/anl/git/satelites/venv/lib/python3.10/site-packages/matplotlib/mpl-data/fonts/ttf'), PosixPath('/home/anl/git/satelites/venv/lib/python3.10/site-packages/matplotlib/mpl-data/fonts/afm'), PosixPath('/home/anl/git/satelites/venv/lib/python3.10/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts')]
INFO:matplotlib.font_manager:Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: Can not load face (unknown file format; error code 0x2)
INFO:matplotlib.font_manager:generated new fontManager
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:mat

In [55]:
import datetime
import os
import sys
import dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) # Add parent directory to path

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig
)

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io

from modules.utils import plot_image

import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(name)s %(threadName)s : %(message)s')


Configuration

In [56]:
dotenv.load_dotenv()

CLIENT_ID = os.getenv("clientID")
CLIENT_SECRET = os.getenv("clientSecret")

if not CLIENT_ID or not CLIENT_SECRET:
    logging.error("Client ID and/or Client Secret not set in environment variables.")
    raise ValueError("Client ID and/or Client Secret not set in environment variables.")

config = SHConfig()
config.instance_id = "test"
config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.save()

In [57]:
# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

# get an authentication token
token = oauth.fetch_token(
    token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
    client_secret=CLIENT_SECRET,
    include_client_id=True
)

DEBUG:requests_oauthlib.oauth2_session:Requesting url https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token using method POST.
DEBUG:requests_oauthlib.oauth2_session:Supplying headers {'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'} and data {'grant_type': 'client_credentials', 'client_id': 'sh-68406116-f980-4851-b489-588bccc733b0', 'client_secret': 'GAKaTt0TxfMH2JgdxMUWo3klbHlMCB6k'}
DEBUG:requests_oauthlib.oauth2_session:Passing through key word arguments {'timeout': None, 'auth': None, 'verify': None, 'proxies': None, 'cert': None}.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): identity.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://identity.dataspace.copernicus.eu:443 "POST /auth/realms/CDSE/protocol/openid-connect/token HTTP/1.1" 200 1721
DEBUG:requests_oauthlib.oauth2_session:Request to fetch token completed with status 200.
DEBUG:requests_oauthlib.oauth2_session:Request url w

In [58]:
bbox = [-87.72171, 17.11848, -87.342682, 17.481674]
start_date = "2020-06-01"
end_date = "2020-08-31"
collection_id = "sentinel-2-l2a"

In [59]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: {
      bands: 3,
      sampleType: "AUTO" // default value - scales the output values from [0,1] to [0,255].
    }
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

In [ ]:
json_request = <copied payload>
"evalscript": evalscript
}

Plotting Betsiboka River in Madagaskar

In [45]:
betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)

resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

Image shape at 60 m resolution: (631, 1047) pixels


In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            DataCollection.SENTINEL2_L1C.define_from("s2l1c", service_url=config.sh_base_url),
            time_interval=("2020-06-12", "2020-06-13"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)

In [53]:
true_color_imgs = request_true_color.get_data()

DEBUG:sentinelhub.download.sentinelhub_client:Sending POST request to https://services.sentinel-hub.com/api/v1/process. Hash of sent request is 52a40d630b187de3c1d28977f102e710
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): services.sentinel-hub.com:443
DEBUG:urllib3.connectionpool:https://services.sentinel-hub.com:443 "POST /api/v1/process HTTP/1.1" 401 222


DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
401 Client Error: Unauthorized for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 401, "reason": "Unauthorized", "message": "You are not authorized! Please provide a valid access token within the header [Authorization: Bearer <accessToken>] of your request.", "code": "COMMON_UNAUTHORIZED"}"